In [1]:
import openslide
from openslide import lowlevel as openslide_ll
from ctypes import c_uint32, POINTER, cast, addressof, c_void_p, byref, c_uint8, sizeof, c_uint16
import numpy as np
from typing import Tuple, Optional
from loguru import logger
import re
import cv2
from multiprocessing import RawArray
import concurrent
from concurrent.futures import ProcessPoolExecutor
from concurrent import futures
from tqdm import tqdm
from pathlib import Path
import dask.array as da
import dask
from dask.distributed import as_completed, futures_of
from dask.distributed import Client, LocalCluster
from functools import partial
from time import time

from tsl8.progress import tqdm_dask
from tsl8.canny import is_background
from tsl8.slide import load_slide, MPPExtractionError

slide_file = "/pathology/camelyon16/training/tumor/tumor_001.tif"
output_path = Path("/data/tsl8_out")
slide_output_dir = output_path / Path(slide_file).stem
slide_output_dir.mkdir(exist_ok=True, parents=True)
check_status = False

patch_size = 512

# target_mpp = 4.0
# level = None  # None means auto select
# patch_to_chunk_multiplier = 2

target_mpp = 0.5
level = 0
patch_to_chunk_multiplier = 16

In [2]:
client = Client(n_workers=32, threads_per_worker=1, memory_limit="64GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 32,Total memory: 1.86 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38295,Workers: 32
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 1.86 TiB
Comm: tcp://127.0.0.1:35449,Total threads: 1
Dashboard: http://127.0.0.1:44751/status,Memory: 59.60 GiB
Nanny: tcp://127.0.0.1:44405,


In [3]:
"""Process a slide and save the features to disk."""
start_time = time()
slide_file = Path(slide_file)

optimize_graph = False

futures_to_cancel = []

# Check if the slide has already been processed
status_file = output_path / "status" / f"{slide_file.stem}.done"
if check_status and status_file.exists():
    logger.info(f"Skipping {slide_file.stem} because it has already been processed")
    raise


def write_status_file(message: str = "done"):
    status_file.parent.mkdir(parents=True, exist_ok=True)
    with status_file.open("w") as f:
        f.write(message)


# Load the slide
try:
    slide = load_slide(
        slide_file, target_mpp=target_mpp, level=level, target_slide_chunk_size=patch_size * patch_to_chunk_multiplier
    )
except MPPExtractionError:
    logger.warning(f"Could not extract MPP for {slide_file}, skipping")
    write_status_file("mpp_extraction_error")
    raise

slide

2023-12-30 16:37:57.348 | DEBUG    | tsl8.slide.readers:make_slide_reader:15 - Using cucim backend for /pathology/camelyon16/training/tumor/tumor_001.tif
2023-12-30 16:37:57.508 | DEBUG    | tsl8.slide.readers.mpp:__call__:24 - MPP successfully extracted using extract_mpp_from_properties: 0.243
2023-12-30 16:37:57.510 | DEBUG    | tsl8.slide:load_slide:25 - Slide has 10 levels with following downsamples: {0: 1.0, 1: 2.0, 2: 4.0, 3: 8.0, 4: 16.0, 5: 32.0, 6: 64.0, 7: 128.0, 8: 256.0, 9: 512.0}
2023-12-30 16:37:57.511 | DEBUG    | tsl8.slide:load_slide:40 - Using level 0 with level_mpp=0.243 for slide_mpp=0.243 and target_mpp=0.500
2023-12-30 16:37:57.513 | DEBUG    | tsl8.slide.readers:make_slide_reader:15 - Using cucim backend for /pathology/camelyon16/training/tumor/tumor_001.tif


dask.array<resize_chunk, shape=(114688, 49152, 3), dtype=uint8, chunksize=(8192, 8192, 3), chunktype=numpy.ndarray>

In [4]:
def reshape_block(block, patch_size=patch_size):
    h, w, c = block.shape
    block = block.reshape(h // patch_size, patch_size, w // patch_size, patch_size, c)

    # Transpose the dimensions to get the tiles in the correct order
    block = block.transpose(0, 2, 1, 3, 4)

    # Reshape again to get a 4D array where each element is a tile
    block = block.reshape(-1, patch_size, patch_size, c)

    return block


def slide_to_chunks(slide, patch_size=patch_size):
    """Convert a slide to a dask array of chunks.

    Returns: array of dask delayed objects, each of which is a chunk (a 4D array of shape (chunksize, patch_size, patch_size, 3))
    """
    k = np.prod(np.array(slide.chunksize[:2]) // np.array((patch_size, patch_size)))
    d = da.blockwise(
        partial(reshape_block, patch_size=patch_size),
        "kijc",
        slide,
        "ijc",
        dtype=slide.dtype,
        new_axes={"k": k},
        adjust_chunks={"i": patch_size, "j": patch_size},
    )
    patches = d.to_delayed().flatten()
    # patches = da.concatenate(
    #     [da.from_delayed(delayed, shape=(k, patch_size, patch_size, 3), dtype=slide.dtype) for delayed in patches]
    # )

    return patches


patches = slide_to_chunks(
    slide, patch_size=patch_size
)  # array of dask delayed objects, each of which is a chunk (a 4D array of shape (n, patch_size, patch_size, 3))

In [5]:
slide_h, slide_w, _ = slide.shape

xs, ys = da.meshgrid(da.arange(slide_w // patch_size), da.arange(slide_h // patch_size))
patch_coords = da.stack([xs, ys], axis=-1) * patch_size
patch_coords = patch_coords.rechunk((patch_to_chunk_multiplier, patch_to_chunk_multiplier, 2))
patch_coords = slide_to_chunks(
    patch_coords, patch_size=1
)  # array of dask delayed objects, each of which is a chunk (a 4D array of shape (n, 1, 1, 2))

In [6]:
def process_block_for_slide(slide_output_dir: Path):
    @dask.delayed
    def process_block(patches, coords):
        """Process a block of patches (perform background rejection, and save to disk)

        Args:
            patches: 4D array of shape (n, patch_size, patch_size, 3)
            coords: 4D array of shape (n, 1, 1, 2)
        """

        # Reject background patches
        patch_mask = is_background(patches)
        patches = patches[~patch_mask]
        coords = coords[~patch_mask].squeeze((1, 2))

        # Save the patches
        for patch, (x, y) in zip(patches, coords):
            patch = cv2.cvtColor(patch, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(slide_output_dir / f"{x:08d}_{y:08d}.jpg"), patch)

        return patches.shape[0]

    return process_block

In [7]:
results = [process_block_for_slide(slide_output_dir)(patch, coord) for patch, coord in zip(patches, patch_coords)]
results = client.persist(results)
for future in tqdm(as_completed(futures_of(results)), total=len(results), desc=f"Processing {slide_file.stem}"):
    pass